##### The cell below is for you to keep track of the libraries used and install those libraries quickly
##### Ensure that the proper library names are used and the syntax of `%pip install PACKAGE_NAME` is followed

In [384]:
#%pip install pandas 
#%pip install matplotlib
# add commented pip installation lines for packages used as shown above for ease of testing
# the line should be of the format %pip install PACKAGE_NAME 

## **DO NOT CHANGE** the filepath variable
##### Instead, create a folder named 'data' in your current working directory and 
##### have the .parquet file inside that. A relative path *must* be used when loading data into pandas

In [439]:
# Can have as many cells as you want for code
import pandas as pd
import numpy as np
filepath = "./data/catB_train.parquet" 
# the initialised filepath MUST be a relative path to a folder named data that contains the parquet file

In [440]:
df = pd.read_parquet(filepath)
df.head().style

,clntnum,race_desc,ctrycode_desc,clttype,stat_flag,min_occ_date,cltdob_fix,cltsex_fix,flg_substandard,flg_is_borderline_standard,flg_is_revised_term,flg_is_rental_flat,flg_has_health_claim,flg_has_life_claim,flg_gi_claim,flg_is_proposal,flg_with_preauthorisation,flg_is_returned_mail,is_consent_to_mail,is_consent_to_email,is_consent_to_call,is_consent_to_sms,is_valid_dm,is_valid_email,is_housewife_retiree,is_sg_pr,is_class_1_2,is_dependent_in_at_least_1_policy,f_ever_declined_la,hh_20,pop_20,hh_size,hh_size_est,annual_income_est,n_months_last_bought_products,flg_latest_being_lapse,flg_latest_being_cancel,recency_lapse,recency_cancel,tot_inforce_pols,tot_cancel_pols,ape_gi_42e115,ape_ltc_1280bf,ape_grp_6fc3e6,ape_grp_de05ae,ape_inv_dcd836,ape_grp_945b5a,ape_grp_6a5788,ape_ltc_43b9d5,ape_grp_9cdedf,ape_lh_d0adeb,ape_grp_1581d7,ape_grp_22decf,ape_lh_507c37,ape_lh_839f8a,ape_inv_e9f316,ape_gi_a10d1b,ape_gi_29d435,ape_grp_caa6ff,ape_grp_fd3bfb,ape_lh_e22a6a,ape_grp_70e1dd,ape_grp_e04c3a,ape_grp_fe5fb8,ape_gi_856320,ape_grp_94baec,ape_gi_058815,ape_grp_e91421,ape_lh_f852af,ape_lh_947b15,ape_32c74c,sumins_gi_42e115,sumins_ltc_1280bf,sumins_grp_6fc3e6,sumins_grp_de05ae,sumins_inv_dcd836,sumins_grp_945b5a,sumins_grp_6a5788,sumins_ltc_43b9d5,sumins_grp_9cdedf,sumins_lh_d0adeb,sumins_grp_1581d7,sumins_grp_22decf,sumins_lh_507c37,sumins_inv_e9f316,sumins_gi_a10d1b,sumins_gi_29d435,sumins_grp_caa6ff,sumins_grp_fd3bfb,sumins_lh_e22a6a,sumins_grp_70e1dd,sumins_grp_e04c3a,sumins_grp_fe5fb8,sumins_gi_856320,sumins_grp_94baec,sumins_gi_058815,sumins_grp_e91421,sumins_lh_f852af,sumins_lh_947b15,sumins_32c74c,prempaid_gi_42e115,prempaid_ltc_1280bf,prempaid_grp_6fc3e6,prempaid_grp_de05ae,prempaid_inv_dcd836,prempaid_grp_945b5a,prempaid_grp_6a5788,prempaid_ltc_43b9d5,prempaid_grp_9cdedf,prempaid_lh_d0adeb,prempaid_grp_1581d7,prempaid_grp_22decf,prempaid_lh_507c37,prempaid_lh_839f8a,prempaid_inv_e9f316,prempaid_gi_a10d1b,prempaid_gi_29d435,prempaid_grp_caa6ff,prempaid_grp_fd3bfb,prempaid_lh_e22a6a,prempaid_grp_70e1dd,prempaid_grp_e04c3a,prempaid_grp_fe5fb8,prempaid_gi_856320,prempaid_grp_94baec,prempaid_gi_058815,prempaid_grp_e91421,prempaid_lh_f852af,prempaid_lh_947b15,prempaid_32c74c,ape_839f8a,ape_e22a6a,ape_d0adeb,ape_c4bda5,ape_ltc,ape_507c37,ape_gi,f_hold_839f8a,f_hold_e22a6a,f_hold_d0adeb,f_hold_c4bda5,f_hold_ltc,f_hold_507c37,f_hold_gi,sumins_839f8a,sumins_e22a6a,sumins_d0adeb,sumins_c4bda5,sumins_ltc,sumins_507c37,sumins_gi,prempaid_839f8a,prempaid_e22a6a,prempaid_d0adeb,prempaid_c4bda5,prempaid_ltc,prempaid_507c37,prempaid_gi,lapse_ape_ltc_1280bf,lapse_ape_grp_6fc3e6,lapse_ape_grp_de05ae,lapse_ape_inv_dcd836,lapse_ape_grp_945b5a,lapse_ape_grp_6a5788,lapse_ape_ltc_43b9d5,lapse_ape_grp_9cdedf,lapse_ape_lh_d0adeb,lapse_ape_grp_1581d7,lapse_ape_grp_22decf,lapse_ape_lh_507c37,lapse_ape_lh_839f8a,lapse_ape_inv_e9f316,lapse_ape_grp_caa6ff,lapse_ape_grp_fd3bfb,lapse_ape_lh_e22a6a,lapse_ape_grp_70e1dd,lapse_ape_grp_e04c3a,lapse_ape_grp_fe5fb8,lapse_ape_grp_94baec,lapse_ape_grp_e91421,lapse_ape_lh_f852af,lapse_ape_lh_947b15,lapse_ape_32c74c,n_months_since_lapse_ltc_1280bf,n_months_since_lapse_grp_6fc3e6,n_months_since_lapse_grp_de05ae,n_months_since_lapse_inv_dcd836,n_months_since_lapse_grp_945b5a,n_months_since_lapse_grp_6a5788,n_months_since_lapse_ltc_43b9d5,n_months_since_lapse_grp_9cdedf,n_months_since_lapse_lh_d0adeb,n_months_since_lapse_grp_1581d7,n_months_since_lapse_grp_22decf,n_months_since_lapse_lh_507c37,n_months_since_lapse_lh_839f8a,n_months_since_lapse_inv_e9f316,n_months_since_lapse_grp_caa6ff,n_months_since_lapse_grp_fd3bfb,n_months_since_lapse_lh_e22a6a,n_months_since_lapse_grp_70e1dd,n_months_since_lapse_grp_e04c3a,n_months_since_lapse_grp_fe5fb8,n_months_since_lapse_grp_94baec,n_months_since_lapse_grp_e91421,n_months_since_lapse_lh_f852af,n_months_since_lapse_lh_947b15,n_months_since_lapse_32c74c,f_ever_bought_839f8a,f_ever_bought_e22a6a,f_ever_bought_d0adeb,f_ever_bought_c4bda5,f_ever_bought_ltc,f_ever_bought_507c37,f_ever_bought_

In [441]:
df["f_purchase_lh"].unique()

array([nan,  1.])

In [387]:
def clean_general_info(df):
    df1 = df.copy()

    # drop clntnum
    df1 = df1.drop("clntnum", axis=1)

    # races: None in race_desc should be replaced as a value. Either unknown or others 
    df1['race_desc'] = df1['race_desc'].fillna("Others")

    # country: None should be replaced with Not Applicable 
    # Combine both Not applicable and unknown country code
    df1['ctrycode_desc'] = df1['ctrycode_desc'].fillna("Not Applicable")

    df1['ctrycode_desc'] = df1['ctrycode_desc'].apply(lambda x: x if x == 'Singapore' else 'Others')

    # min_occ_date: calculate the duration this customer has been with singlife 
    # convert to datetime
    df1['min_occ_date'] = pd.to_datetime(df1['min_occ_date'], errors='coerce')
    years_with_singlife = (pd.to_datetime('now') - df1['min_occ_date']).dt.days / 365.25
    mean_duration = years_with_singlife.median() 
    df1['years_with_company'] = years_with_singlife.fillna(mean_duration).astype(int)
    df1.drop('min_occ_date', axis=1, inplace=True)

    # cltdob_fix: calculate age
    df1['cltdob_fix'] = pd.to_datetime(df1['cltdob_fix'], errors='coerce')
    age_in_years = (pd.to_datetime('now') - df1['cltdob_fix']).dt.days / 365.25
    mean_age = age_in_years.mean()
    df1['age'] = age_in_years.fillna(mean_age).astype(int) # fill with mean age
    # dropping DOB column 
    df1.drop('cltdob_fix', axis=1, inplace=True)

    # None value in gender 
    df1['cltsex_fix'].unique()
    # None value in gender: only a few, so replace by default: male 
    df1['cltsex_fix'] = df1['cltsex_fix'].fillna('Male')

    return df1 
df2 = clean_general_info(df)
df2.head().style

,race_desc,ctrycode_desc,clttype,stat_flag,cltsex_fix,flg_substandard,flg_is_borderline_standard,flg_is_revised_term,flg_is_rental_flat,flg_has_health_claim,flg_has_life_claim,flg_gi_claim,flg_is_proposal,flg_with_preauthorisation,flg_is_returned_mail,is_consent_to_mail,is_consent_to_email,is_consent_to_call,is_consent_to_sms,is_valid_dm,is_valid_email,is_housewife_retiree,is_sg_pr,is_class_1_2,is_dependent_in_at_least_1_policy,f_ever_declined_la,hh_20,pop_20,hh_size,hh_size_est,annual_income_est,n_months_last_bought_products,flg_latest_being_lapse,flg_latest_being_cancel,recency_lapse,recency_cancel,tot_inforce_pols,tot_cancel_pols,ape_gi_42e115,ape_ltc_1280bf,ape_grp_6fc3e6,ape_grp_de05ae,ape_inv_dcd836,ape_grp_945b5a,ape_grp_6a5788,ape_ltc_43b9d5,ape_grp_9cdedf,ape_lh_d0adeb,ape_grp_1581d7,ape_grp_22decf,ape_lh_507c37,ape_lh_839f8a,ape_inv_e9f316,ape_gi_a10d1b,ape_gi_29d435,ape_grp_caa6ff,ape_grp_fd3bfb,ape_lh_e22a6a,ape_grp_70e1dd,ape_grp_e04c3a,ape_grp_fe5fb8,ape_gi_856320,ape_grp_94baec,ape_gi_058815,ape_grp_e91421,ape_lh_f852af,ape_lh_947b15,ape_32c74c,sumins_gi_42e115,sumins_ltc_1280bf,sumins_grp_6fc3e6,sumins_grp_de05ae,sumins_inv_dcd836,sumins_grp_945b5a,sumins_grp_6a5788,sumins_ltc_43b9d5,sumins_grp_9cdedf,sumins_lh_d0adeb,sumins_grp_1581d7,sumins_grp_22decf,sumins_lh_507c37,sumins_inv_e9f316,sumins_gi_a10d1b,sumins_gi_29d435,sumins_grp_caa6ff,sumins_grp_fd3bfb,sumins_lh_e22a6a,sumins_grp_70e1dd,sumins_grp_e04c3a,sumins_grp_fe5fb8,sumins_gi_856320,sumins_grp_94baec,sumins_gi_058815,sumins_grp_e91421,sumins_lh_f852af,sumins_lh_947b15,sumins_32c74c,prempaid_gi_42e115,prempaid_ltc_1280bf,prempaid_grp_6fc3e6,prempaid_grp_de05ae,prempaid_inv_dcd836,prempaid_grp_945b5a,prempaid_grp_6a5788,prempaid_ltc_43b9d5,prempaid_grp_9cdedf,prempaid_lh_d0adeb,prempaid_grp_1581d7,prempaid_grp_22decf,prempaid_lh_507c37,prempaid_lh_839f8a,prempaid_inv_e9f316,prempaid_gi_a10d1b,prempaid_gi_29d435,prempaid_grp_caa6ff,prempaid_grp_fd3bfb,prempaid_lh_e22a6a,prempaid_grp_70e1dd,prempaid_grp_e04c3a,prempaid_grp_fe5fb8,prempaid_gi_856320,prempaid_grp_94baec,prempaid_gi_058815,prempaid_grp_e91421,prempaid_lh_f852af,prempaid_lh_947b15,prempaid_32c74c,ape_839f8a,ape_e22a6a,ape_d0adeb,ape_c4bda5,ape_ltc,ape_507c37,ape_gi,f_hold_839f8a,f_hold_e22a6a,f_hold_d0adeb,f_hold_c4bda5,f_hold_ltc,f_hold_507c37,f_hold_gi,sumins_839f8a,sumins_e22a6a,sumins_d0adeb,sumins_c4bda5,sumins_ltc,sumins_507c37,sumins_gi,prempaid_839f8a,prempaid_e22a6a,prempaid_d0adeb,prempaid_c4bda5,prempaid_ltc,prempaid_507c37,prempaid_gi,lapse_ape_ltc_1280bf,lapse_ape_grp_6fc3e6,lapse_ape_grp_de05ae,lapse_ape_inv_dcd836,lapse_ape_grp_945b5a,lapse_ape_grp_6a5788,lapse_ape_ltc_43b9d5,lapse_ape_grp_9cdedf,lapse_ape_lh_d0adeb,lapse_ape_grp_1581d7,lapse_ape_grp_22decf,lapse_ape_lh_507c37,lapse_ape_lh_839f8a,lapse_ape_inv_e9f316,lapse_ape_grp_caa6ff,lapse_ape_grp_fd3bfb,lapse_ape_lh_e22a6a,lapse_ape_grp_70e1dd,lapse_ape_grp_e04c3a,lapse_ape_grp_fe5fb8,lapse_ape_grp_94baec,lapse_ape_grp_e91421,lapse_ape_lh_f852af,lapse_ape_lh_947b15,lapse_ape_32c74c,n_months_since_lapse_ltc_1280bf,n_months_since_lapse_grp_6fc3e6,n_months_since_lapse_grp_de05ae,n_months_since_lapse_inv_dcd836,n_months_since_lapse_grp_945b5a,n_months_since_lapse_grp_6a5788,n_months_since_lapse_ltc_43b9d5,n_months_since_lapse_grp_9cdedf,n_months_since_lapse_lh_d0adeb,n_months_since_lapse_grp_1581d7,n_months_since_lapse_grp_22decf,n_months_since_lapse_lh_507c37,n_months_since_lapse_lh_839f8a,n_months_since_lapse_inv_e9f316,n_months_since_lapse_grp_caa6ff,n_months_since_lapse_grp_fd3bfb,n_months_since_lapse_lh_e22a6a,n_months_since_lapse_grp_70e1dd,n_months_since_lapse_grp_e04c3a,n_months_since_lapse_grp_fe5fb8,n_months_since_lapse_grp_94baec,n_months_since_lapse_grp_e91421,n_months_since_lapse_lh_f852af,n_months_since_lapse_lh_947b15,n_months_since_lapse_32c74c,f_ever_bought_839f8a,f_ever_bought_e22a6a,f_ever_bought_d0adeb,f_ever_bought_c4bda5,f_ever_bought_ltc,f_ever_bought_507c37,f_ever_bought_gi,n_months_last_bought_839f8a,n

In [388]:
def clean_up_risk_and_status(df):
    df1 = df.copy()
    # clean up float to int 
    columns_to_convert = ['flg_substandard', 'flg_is_borderline_standard', 'flg_is_revised_term',
       'flg_is_rental_flat', 'flg_has_health_claim', 'flg_has_life_claim',
       'flg_gi_claim', 'flg_is_proposal', 'flg_with_preauthorisation',
       'flg_is_returned_mail']
    for column in columns_to_convert:
        df1[column] = df1[column].fillna(0).astype(float).astype(int)

    # New feature 1: has health / life / general insurance -> there is no need to distinguish them as the Qn ask for general purcahase too 

    # 1. Number of Insurance purchased
    df1['num_total_claims'] = (df1['flg_has_health_claim'] + df1['flg_has_life_claim'] + df1['flg_gi_claim']).fillna(0)

    # # 2. whether they purchased insurance (bought at least once)
    # df1['flg_health_or_life_claim'] = df1['flg_has_health_claim'] | df1['flg_has_life_claim'] | df1['flg_gi_claim']

    # 3. combine risk together: substandard risk -> level 3, boarderline -> level 2, standard -> level 1 
    conditions = [
        df1['flg_substandard'] == 1,
        df1['flg_is_borderline_standard'] == 1,
        ~(df1['flg_substandard'] == 1) & ~(df1['flg_is_borderline_standard'] == 1)  # Standard risk
    ]

    values = [3, 2, 1]

    # Create a new column 'risk_level' based on the conditions, if na means standard 
    df1['risk_level'] = np.select(conditions, values, default=1)

    # 4. dropping used columns 
    df1.drop(columns = ['flg_substandard','flg_is_borderline_standard','flg_has_health_claim','flg_has_life_claim','flg_gi_claim'],inplace=True)
    
    return df1


In [389]:
df2 = clean_up_risk_and_status(df2)
df2.head()

,race_desc,ctrycode_desc,clttype,stat_flag,cltsex_fix,flg_is_revised_term,flg_is_rental_flat,flg_is_proposal,flg_with_preauthorisation,flg_is_returned_mail,...,recency_giclaim_unsuccess,flg_gi_claim_29d435_ever,flg_gi_claim_058815_ever,flg_gi_claim_42e115_ever,flg_gi_claim_856320_ever,f_purchase_lh,years_with_company,age,num_total_claims,risk_level
19550,Chinese,Singapore,P,ACTIVE,Female,0,0,0,0,0,...,None,None,None,None,None,NaN,6,49,0,1
4600,Chinese,Singapore,P,ACTIVE,Male,0,0,0,0,0,...,None,None,None,None,None,NaN,16,44,0,1
13337,Others,Singapore,P,ACTIVE,Male,0,0,0,0,0,...,None,None,None,None,None,NaN,4,48,0,1
15074,Chinese,Singapore,P,ACTIVE,Female,0,0,0,0,0,...,None,None,None,None,None,NaN,2,47,0,1
19724,Chinese,Singapore,P,ACTIVE,Female,0,0,0,0,0,...,None,None,None,None,None,NaN,5,28,0,1


In [390]:
# Number of Insurance purchased
# df1['num_total_claims'] = (df1['flg_has_health_claim'] + df1['flg_has_life_claim'] + df1['flg_gi_claim']).fillna(0)

In [391]:
# # combine risk together: substandard risk -> level 3, boarderline -> level 2, standard -> level 1 
# conditions = [
#     df1['flg_substandard'] == 1,
#     df1['flg_is_borderline_standard'] == 1,
#     ~(df1['flg_substandard'] == 1) & ~(df1['flg_is_borderline_standard'] == 1)  # Standard risk
# ]

# values = [3, 2, 1]

# # Create a new column 'risk_level' based on the conditions, if na means standard 
# df1['risk_level'] = np.select(conditions, values, default=1)

# df1.drop(columns = ['flg_substandard','flg_is_borderline_standard','flg_has_health_claim','flg_has_life_claim','flg_gi_claim'],inplace=True)


In [392]:
def clean_up_household_info(df):

    df1 = df.copy()

    household_col = ['is_housewife_retiree', 'is_sg_pr', 'is_class_1_2',
       'is_dependent_in_at_least_1_policy', 'hh_20',
       'pop_20']
       
    for column in household_col:
        df1[column] = df1[column].fillna(0).astype(float).astype(int)

    # annual income set 
    df1['annual_income_est'] = df1['annual_income_est'].fillna("F.UNKNOWN")
    income_mapping = {
        'F.UNKNOWN': 1,
        'E.BELOW30K': 2,
        'D.30K-60K': 3,
        'C.60K-100K': 4,
        'B.100K-200K': 5,
        'A.ABOVE200K': 6
    }

    # Apply ordinal encoding
    df1['annual_income_encoded'] = df1['annual_income_est'].map(income_mapping)

    # hh_size_est: level 0 to level 5  
    df1['hh_size_est'] = df1['hh_size_est'].fillna("0")
    hh_size_mapping = {
        '1': 1,
        '2': 2,
        '3': 3,
        '4': 4,
        '>4': 5,
        '0': 0
    }

    # Apply ordinal encoding
    df1['hh_size_est_encoded'] = df1['hh_size_est'].map(hh_size_mapping)

    # dropping columns 
    df1.drop(columns=['hh_20','pop_20','hh_size','hh_size_est','annual_income_est'],inplace=True)

    return df1

In [393]:
df2 = clean_up_household_info(df2)
df2.head()

,race_desc,ctrycode_desc,clttype,stat_flag,cltsex_fix,flg_is_revised_term,flg_is_rental_flat,flg_is_proposal,flg_with_preauthorisation,flg_is_returned_mail,...,flg_gi_claim_058815_ever,flg_gi_claim_42e115_ever,flg_gi_claim_856320_ever,f_purchase_lh,years_with_company,age,num_total_claims,risk_level,annual_income_encoded,hh_size_est_encoded
19550,Chinese,Singapore,P,ACTIVE,Female,0,0,0,0,0,...,None,None,None,NaN,6,49,0,1,4,1
4600,Chinese,Singapore,P,ACTIVE,Male,0,0,0,0,0,...,None,None,None,NaN,16,44,0,1,3,3
13337,Others,Singapore,P,ACTIVE,Male,0,0,0,0,0,...,None,None,None,NaN,4,48,0,1,6,3
15074,Chinese,Singapore,P,ACTIVE,Female,0,0,0,0,0,...,None,None,None,NaN,2,47,0,1,5,4
19724,Chinese,Singapore,P,ACTIVE,Female,0,0,0,0,0,...,None,None,None,NaN,5,28,0,1,2,5


In [394]:
# # convert to boolean: is_housewife_retiree, is_sg_pr, is_class_1_2, is_dependent_in_at_least_1_policy 
# col = ['is_housewife_retiree', 'is_sg_pr', 'is_class_1_2', 'is_dependent_in_at_least_1_policy']
# for column in col:
#     df1[column] = df1[column].fillna(0).astype(float).astype(int)

In [395]:
# # annual income set 
# df1['annual_income_est'] = df1['annual_income_est'].fillna("F.UNKNOWN")
# income_mapping = {
#     'F. UNKNOWN': 1,
#     'E.BELOW30K': 2,
#     'D.30K-60K': 3,
#     'C.60K-100K': 4,
#     'B.100K-200K': 5,
#     'A.ABOVE200K': 6
# }

# # Apply ordinal encoding
# df1['annual_income_encoded'] = df1['annual_income_est'].map(income_mapping)

In [396]:
# df1['hh_size_est'] = df1['hh_size_est'].fillna("0")

# hh_size_mapping = {
#     '1': 1,
#     '2': 2,
#     '3': 3,
#     '4': 4,
#     '>4': 5,
#     '0': 0
# }

# # Apply ordinal encoding
# df1['hh_size_est_encoded'] = df1['hh_size_est'].map(hh_size_mapping)

# df1.drop(columns=['hh_20','pop_20','hh_size','hh_size_est','annual_income_est'],inplace=True)

In [397]:
def clean_consent(df):
    df1 = df.copy()
    consent_col = ['is_consent_to_mail', 'is_consent_to_email', 'is_consent_to_call',
        'is_consent_to_sms', 'is_valid_dm', 'is_valid_email']
    for column in consent_col:
        df1[column] = df1[column].fillna(0).astype(float).astype(int) # na can just assume they don't consent 

    conditions = [
        (df1['is_consent_to_call'] == 1),
        (df1['is_consent_to_email'] == 1) & (df1['is_valid_email'] == 1),
        (df1['is_consent_to_mail'] == 1) & (df1['is_valid_dm'] == 1),
        (df1['is_consent_to_sms'] == 1)
    ]

    # Define the corresponding consent levels
    values = [5, 1, 1, 1]

    # Create the 'consent_level' column by summing the values based on the conditions
    df1['consent_level'] = np.sum(np.column_stack(conditions) * np.array(values), axis=1)

    # drop old consent columns 
    df1.drop(columns=['is_consent_to_mail','is_consent_to_email','is_consent_to_call','is_consent_to_sms','is_valid_dm','is_valid_email'],inplace=True)

    return df1

In [398]:
df2 = clean_consent(df2)
df2.head()

,race_desc,ctrycode_desc,clttype,stat_flag,cltsex_fix,flg_is_revised_term,flg_is_rental_flat,flg_is_proposal,flg_with_preauthorisation,flg_is_returned_mail,...,flg_gi_claim_42e115_ever,flg_gi_claim_856320_ever,f_purchase_lh,years_with_company,age,num_total_claims,risk_level,annual_income_encoded,hh_size_est_encoded,consent_level
19550,Chinese,Singapore,P,ACTIVE,Female,0,0,0,0,0,...,None,None,NaN,6,49,0,1,4,1,0
4600,Chinese,Singapore,P,ACTIVE,Male,0,0,0,0,0,...,None,None,NaN,16,44,0,1,3,3,0
13337,Others,Singapore,P,ACTIVE,Male,0,0,0,0,0,...,None,None,NaN,4,48,0,1,6,3,2
15074,Chinese,Singapore,P,ACTIVE,Female,0,0,0,0,0,...,None,None,NaN,2,47,0,1,5,4,0
19724,Chinese,Singapore,P,ACTIVE,Female,0,0,0,0,0,...,None,None,NaN,5,28,0,1,2,5,3


In [399]:
# #consent_col = ['is_consent_to_call', 'is_consent_to_email', 'is_consent_to_mail', 'is_consent_to_sms']
# consent_col = ['is_consent_to_mail', 'is_consent_to_email', 'is_consent_to_call',
#         'is_consent_to_sms', 'is_valid_dm', 'is_valid_email']

# for column in consent_col:
#     df1[column] = df1[column].fillna(0).astype(float).astype(int) # na can just assume they don't consent 

# conditions = [
#     (df1['is_consent_to_call'] == 1),
#     (df1['is_consent_to_email'] == 1) & (df1['is_valid_email'] == 1),
#     (df1['is_consent_to_mail'] == 1) & (df1['is_valid_dm'] == 1),
#     (df1['is_consent_to_sms'] == 1)
# ]

# values = [5, 1, 1, 1]

# # Create the 'consent_level' column by summing the values based on the conditions
# df1['consent_level'] = np.sum(np.column_stack(conditions) * np.array(values), axis=1)
# df1.drop(columns=['is_consent_to_mail','is_consent_to_email','is_consent_to_call','is_consent_to_sms','is_valid_dm','is_valid_email'],inplace=True)

In [400]:
def clean_policy(df):
    df1 = df.copy()
    # should replace nan with 0 
    df1['f_ever_declined_la'] = df1['f_ever_declined_la'].fillna(0)

    # if nan, assume no cancellation / lapse happened, set recency_xxx to be 9999 
    df1["recency_lapse"] = df1["recency_lapse"].fillna(0)
    df1["recency_cancel"] = df1["recency_cancel"].fillna(0)


    # compute avg cancel rate 
    df1['avg_cancel_rate'] = (df1['tot_cancel_pols'] / (df1['tot_cancel_pols'] + df1['tot_inforce_pols'])).fillna(0)

    df1['f_ever_declined_la'] = df1['f_ever_declined_la'].fillna(0)


    df1 = df1.drop(columns=['flg_latest_being_lapse', 'flg_latest_being_cancel'], axis=1)
    return df1

In [401]:
df2 = clean_policy(df2)
df2.head()

,race_desc,ctrycode_desc,clttype,stat_flag,cltsex_fix,flg_is_revised_term,flg_is_rental_flat,flg_is_proposal,flg_with_preauthorisation,flg_is_returned_mail,...,flg_gi_claim_856320_ever,f_purchase_lh,years_with_company,age,num_total_claims,risk_level,annual_income_encoded,hh_size_est_encoded,consent_level,avg_cancel_rate
19550,Chinese,Singapore,P,ACTIVE,Female,0,0,0,0,0,...,None,NaN,6,49,0,1,4,1,0,0.0
4600,Chinese,Singapore,P,ACTIVE,Male,0,0,0,0,0,...,None,NaN,16,44,0,1,3,3,0,0.0
13337,Others,Singapore,P,ACTIVE,Male,0,0,0,0,0,...,None,NaN,4,48,0,1,6,3,2,0.0
15074,Chinese,Singapore,P,ACTIVE,Female,0,0,0,0,0,...,None,NaN,2,47,0,1,5,4,0,0.0
19724,Chinese,Singapore,P,ACTIVE,Female,0,0,0,0,0,...,None,NaN,5,28,0,1,2,5,3,0.0


In [402]:
# df1['f_ever_declined_la'] = df1['f_ever_declined_la'].fillna(0)

# df1["recency_lapse"] = df1["recency_lapse"].fillna(0)
# df1["recency_cancel"] = df1["recency_cancel"].fillna(0)

# df1 = df1.drop(columns=['flg_latest_being_lapse', 'flg_latest_being_cancel'], axis=1)

# df1['tot_cancel_pols'] = df1['tot_cancel_pols'].fillna(0)

# # df1['avg_purchase_frequency'] = (df1['tot_cancel_pols'] + df1['tot_inforce_pols']) / df1['years_with_company'] # division by 0????
# # df1['avg_cancel_rate'] = df1['tot_cancel_pols'] / (df1['tot_cancel_pols'] + df1['tot_inforce_pols']) 

In [403]:
def clean_others(df): 
    df1 = df.copy()
    last_bought_months = df1.filter(regex='n_months_last_bought_.*', axis=1).columns

    for column in last_bought_months:
        df1[column] = df1[column].fillna(0).astype(float).astype(int)


    df1[last_bought_months] = df1[last_bought_months].replace(9999, 0)

    df1['n_months_last_bought_products'] = df1['n_months_last_bought_products'].fillna(0)

    df1['Combined_List_Bought'] = df1[last_bought_months].apply(lambda row: sorted([val for val in row.tolist() if val != 0]), axis=1)
    df1['Bought_Gap_List'] = df1['Combined_List_Bought'].apply(lambda sorted_list: [b - a for a, b in zip(sorted_list[:-1], sorted_list[1:])])
    df1['Average_Bought_Gap'] = df1['Bought_Gap_List'].apply(lambda gaps: np.mean(gaps) if gaps else 0)

    df1['Average_Bought_Gap'][df1['Average_Bought_Gap'] !=0]

    df1['Exceed Average Gap'] = df1['Average_Bought_Gap'] <= df1['n_months_last_bought_products']
    sum(df1['Exceed Average Gap'])

    df1['Exceed Average Gap with Purchase'] = (df1['Average_Bought_Gap'] <= df1['n_months_last_bought_products']) & (df1['Average_Bought_Gap'] != 0)
    sum(df1['Exceed Average Gap with Purchase'])

    df1['Average_Bought_Gap'][df1['Average_Bought_Gap']>=36]

    bins = [-float('inf'), 3, 6, 12, 24, 36, 60, 120, float('inf')]
    labels = ['<3 months', '3-6 months', '6 months - 1 year', '1-2 years', '2-3 years', '3-5 years', '5-10 years', '> 10 years']

    df1['Months_Last_Bought_Category'] = pd.cut(df1['n_months_last_bought_products'], bins=bins, labels=labels)

    df1['Months_Last_Bought_Category']

    column_patterns = ['ape_', 'sumins_', 'prempaid_']
    regex_pattern = '^(' + '|'.join(column_patterns) + ')'
    claimed_amount = df1.filter(regex=regex_pattern, axis=1).columns

    df1['Claimed_Large_Amount'] = df1[claimed_amount].ge(50000).any(axis=1)
    sum(df1['Claimed_Large_Amount'])

    column_patterns = ['lapse_']
    regex_pattern = '^(' + '|'.join(column_patterns) + ')'
    lapsed_amount = df1.filter(regex=regex_pattern, axis=1).columns

    df1['Lapsed_Large_Amount'] = df1[lapsed_amount].ge(200).any(axis=1)
    sum(df1['Lapsed_Large_Amount'])

    last_lapse_months =  df1.filter(regex='n_months_since_lapse.*', axis=1).columns


    #df1[last_lapse_months] = df1[last_lapse_months].replace('9999', 0)
    for column in last_lapse_months:
        df1[column] = df1[column].fillna(0).astype(float).astype(int)


    df1[last_lapse_months] = df1[last_lapse_months].replace(9999, 0)


    df1['Combined_List_Lapse'] = df1[last_lapse_months].apply(lambda row: sorted([val for val in row.tolist() if val != 0]), axis=1)
    df1['Lapse_Gap_List'] = df1['Combined_List_Lapse'].apply(lambda sorted_list: [b - a for a, b in zip(sorted_list[:-1], sorted_list[1:])])
    df1['Average_Lapse_Gap'] = df1['Lapse_Gap_List'].apply(lambda gaps: np.mean(gaps) if gaps else 0)

    columns_to_drop = []

    column_patterns = ['f_ever_bought']
    regex_pattern = '^(' + '|'.join(column_patterns) + ')'
    ever_bought = df1.filter(regex=regex_pattern, axis=1).columns

    columns_to_drop.extend(last_bought_months)
    columns_to_drop.extend(claimed_amount)
    columns_to_drop.extend(last_lapse_months)
    columns_to_drop.extend(lapsed_amount)
    columns_to_drop.extend(ever_bought)
    columns_to_drop.extend(['Combined_List_Lapse','Lapse_Gap_List','Combined_List_Bought', 'Bought_Gap_List'])
    df1.drop(columns=columns_to_drop, inplace=True)


    return df1
    


In [404]:
df2 = clean_others(df2)
df2.head()

,race_desc,ctrycode_desc,clttype,stat_flag,cltsex_fix,flg_is_revised_term,flg_is_rental_flat,flg_is_proposal,flg_with_preauthorisation,flg_is_returned_mail,...,hh_size_est_encoded,consent_level,avg_cancel_rate,Average_Bought_Gap,Exceed Average Gap,Exceed Average Gap with Purchase,Months_Last_Bought_Category,Claimed_Large_Amount,Lapsed_Large_Amount,Average_Lapse_Gap
19550,Chinese,Singapore,P,ACTIVE,Female,0,0,0,0,0,...,1,0,0.0,1.25,False,False,<3 months,False,False,0.0
4600,Chinese,Singapore,P,ACTIVE,Male,0,0,0,0,0,...,3,0,0.0,47.50,False,False,3-5 years,False,True,0.0
13337,Others,Singapore,P,ACTIVE,Male,0,0,0,0,0,...,3,2,0.0,0.00,True,False,3-5 years,False,False,0.0
15074,Chinese,Singapore,P,ACTIVE,Female,0,0,0,0,0,...,4,0,0.0,0.00,True,False,1-2 years,False,False,0.0
19724,Chinese,Singapore,P,ACTIVE,Female,0,0,0,0,0,...,5,3,0.0,0.00,True,False,5-10 years,True,False,0.0


In [405]:
def f_hold_columns(df):
    df1 = df.copy()
    f_hold_columns = df1.filter(regex='f_hold_.*').columns

    for col in f_hold_columns:
        df1[col] = df1[col].astype(int)

    df1["f_hold"] = df1[f_hold_columns].sum(axis=1)
    df1.drop(columns=f_hold_columns, inplace=True)
    return df1 


In [406]:
df2 = f_hold_columns(df2)
df2.head()

,race_desc,ctrycode_desc,clttype,stat_flag,cltsex_fix,flg_is_revised_term,flg_is_rental_flat,flg_is_proposal,flg_with_preauthorisation,flg_is_returned_mail,...,consent_level,avg_cancel_rate,Average_Bought_Gap,Exceed Average Gap,Exceed Average Gap with Purchase,Months_Last_Bought_Category,Claimed_Large_Amount,Lapsed_Large_Amount,Average_Lapse_Gap,f_hold
19550,Chinese,Singapore,P,ACTIVE,Female,0,0,0,0,0,...,0,0.0,1.25,False,False,<3 months,False,False,0.0,2
4600,Chinese,Singapore,P,ACTIVE,Male,0,0,0,0,0,...,0,0.0,47.50,False,False,3-5 years,False,True,0.0,0
13337,Others,Singapore,P,ACTIVE,Male,0,0,0,0,0,...,2,0.0,0.00,True,False,3-5 years,False,False,0.0,0
15074,Chinese,Singapore,P,ACTIVE,Female,0,0,0,0,0,...,0,0.0,0.00,True,False,1-2 years,False,False,0.0,0
19724,Chinese,Singapore,P,ACTIVE,Female,0,0,0,0,0,...,3,0.0,0.00,True,False,5-10 years,True,False,0.0,1


In [407]:
# print(df1.shape)
# df1.head().style

In [408]:
def hlthclaim(df):
    df1 = df.copy()
    pattern = r'flg_(hlthclaim|gi_claim)_[a-z0-9]+_ever'
    filtered_cols = df.filter(regex=pattern).columns

    #print(filtered_cols)

    df1[filtered_cols] = df1[filtered_cols].fillna(0)
    df1['flg_hlthclaim_ever'] = df1[filtered_cols].sum(axis=1)

    df1.drop(columns=filtered_cols, inplace=True)
    return df1


In [409]:
df2 = hlthclaim(df2)
df2.head()

,race_desc,ctrycode_desc,clttype,stat_flag,cltsex_fix,flg_is_revised_term,flg_is_rental_flat,flg_is_proposal,flg_with_preauthorisation,flg_is_returned_mail,...,avg_cancel_rate,Average_Bought_Gap,Exceed Average Gap,Exceed Average Gap with Purchase,Months_Last_Bought_Category,Claimed_Large_Amount,Lapsed_Large_Amount,Average_Lapse_Gap,f_hold,flg_hlthclaim_ever
19550,Chinese,Singapore,P,ACTIVE,Female,0,0,0,0,0,...,0.0,1.25,False,False,<3 months,False,False,0.0,2,0.0
4600,Chinese,Singapore,P,ACTIVE,Male,0,0,0,0,0,...,0.0,47.50,False,False,3-5 years,False,True,0.0,0,0.0
13337,Others,Singapore,P,ACTIVE,Male,0,0,0,0,0,...,0.0,0.00,True,False,3-5 years,False,False,0.0,0,0.0
15074,Chinese,Singapore,P,ACTIVE,Female,0,0,0,0,0,...,0.0,0.00,True,False,1-2 years,False,False,0.0,0,0.0
19724,Chinese,Singapore,P,ACTIVE,Female,0,0,0,0,0,...,0.0,0.00,True,False,5-10 years,True,False,0.0,1,0.0


In [410]:
# print(df1.shape)
# df1.head().style

In [411]:
temp = df2

In [412]:
def sum_and_drop_columns(df, prefix, new_column_name):
    df1 = df.copy()
    filtered_columns = df1.filter(regex=f'^{prefix}', axis=1)
    sumins_columns = filtered_columns.columns
    df1[new_column_name] = filtered_columns.sum(axis=1)
    df1.drop(columns=sumins_columns, inplace=True)
    return df1

# # ape 
# df2 = sum_and_drop_columns(df2, 'ape', 'total_ape_purchased')
# # sumins 
# df2 = sum_and_drop_columns(df2, 'sumins', 'total_sumins_purchased')
# # prempad 
# df2 = sum_and_drop_columns(df2, 'prempaid', 'total_prempaid_purchased')

In [413]:
temp.equals(df2)

True

In [414]:
# import pandas as pd
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.compose import ColumnTransformer
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import StandardScaler, OrdinalEncoder
# from xgboost import XGBClassifier
# from sklearn.metrics import classification_report, confusion_matrix
# from imblearn.over_sampling import SMOTE
# from imblearn.pipeline import Pipeline
# from sklearn.metrics import f1_score


# df = df1
# col_to_target = ['f_purchase_lh']
# for col in col_to_target:
#     df1[col] = df1[col].fillna(0)

# # Separate features and target
# X = df.drop('f_purchase_lh', axis=1)
# y = df['f_purchase_lh']

# # Identify categorical and numerical columns
# categorical_cols = X.select_dtypes(include=['object', 'category']).columns
# numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns

# # Create a preprocessing pipeline
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', SimpleImputer(strategy='mean'), numerical_cols),
#         ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), categorical_cols)
#     ])

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Create a pipeline with preprocessing, SMOTE, and XGBoost Classifier
# model = Pipeline([
#     ('preprocessor', preprocessor),
#     ('scaler', StandardScaler()),
#     ('smote', SMOTE(random_state=42)),
#     ('classifier', XGBClassifier(random_state=42))
# ])

# # Define the hyperparameter grid for XGBoost
# param_grid = {
#     'classifier__learning_rate': [0.01, 0.1, 0.2],
#     'classifier__max_depth': [3, 5, 7],
#     'classifier__n_estimators': [50, 100, 200],
#     'classifier__subsample': [0.8, 0.9, 1.0],
#     'classifier__colsample_bytree': [0.8, 0.9, 1.0],
# }

# # Use Grid Search with cross-validation
# grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='f1', n_jobs=-1)
# grid_search.fit(X_train, y_train)

# # Print the best hyperparameters
# print("Best Hyperparameters:", grid_search.best_params_)

# # Evaluate the model on the test set using the best hyperparameters
# best_model = grid_search.best_estimator_
# y_pred = best_model.predict(X_test)

# # Evaluation
# print(confusion_matrix(y_test, y_pred))
# print(classification_report(y_test, y_pred))

# f1 = f1_score(y_test, y_pred)
# print("F1 Score: ", f1)


In [415]:
#pip install catboost

In [416]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.metrics import f1_score
import joblib

df = df2
col_to_target = ['f_purchase_lh']
for col in col_to_target:
    df[col] = df[col].fillna(0)

# Separate features and target
X = df.drop('f_purchase_lh', axis=1)
y = df['f_purchase_lh']

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object', 'category']).columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns

# Create a preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numerical_cols),
        ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), categorical_cols)
    ])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_test.shape)
print(type(X_test))

# Create a pipeline with preprocessing, SMOTE, and CatBoost Classifier
model = Pipeline([
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('classifier', CatBoostClassifier(random_state=42, verbose=0))
])

# Define the hyperparameter grid for CatBoost
param_grid = {
    'classifier__learning_rate': [0.01, 0.1, 0.2],
    'classifier__depth': [3, 5, 7],
    'classifier__iterations': [50, 100, 200],
    'classifier__subsample': [0.8, 0.9, 1.0],
    'classifier__colsample_bylevel': [0.8, 0.9, 1.0],
}

# Use Grid Search with cross-validation
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='f1', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Evaluate the model on the test set using the best hyperparameters
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Evaluation
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

f1 = f1_score(y_test, y_pred)
print("F1 Score: ", f1)

(3599, 61)
<class 'pandas.core.frame.DataFrame'>


Best Hyperparameters: {'classifier__colsample_bylevel': 1.0, 'classifier__depth': 7, 'classifier__iterations': 200, 'classifier__learning_rate': 0.01, 'classifier__subsample': 0.8}
[[3343  105]
 [ 105   46]]
              precision    recall  f1-score   support

         0.0       0.97      0.97      0.97      3448
         1.0       0.30      0.30      0.30       151

    accuracy                           0.94      3599
   macro avg       0.64      0.64      0.64      3599
weighted avg       0.94      0.94      0.94      3599

F1 Score:  0.304635761589404


In [420]:
y_pred

array([0., 0., 0., ..., 0., 0., 0.])

In [421]:
X_test

,race_desc,ctrycode_desc,clttype,stat_flag,cltsex_fix,flg_is_revised_term,flg_is_rental_flat,flg_is_proposal,flg_with_preauthorisation,flg_is_returned_mail,...,avg_cancel_rate,Average_Bought_Gap,Exceed Average Gap,Exceed Average Gap with Purchase,Months_Last_Bought_Category,Claimed_Large_Amount,Lapsed_Large_Amount,Average_Lapse_Gap,f_hold,flg_hlthclaim_ever
2397,Others,Singapore,G,ACTIVE,Male,0,0,0,0,0,...,0.0,0.000000,True,False,1-2 years,True,False,0.0,2,0.0
19035,Others,Singapore,G,ACTIVE,Male,0,0,0,0,0,...,0.0,0.000000,True,False,6 months - 1 year,False,False,0.0,0,0.0
19957,Chinese,Singapore,P,ACTIVE,Female,0,0,0,0,0,...,0.0,8.000000,False,False,<3 months,True,False,0.0,3,0.0
16719,Chinese,Others,P,ACTIVE,Female,0,0,0,0,0,...,0.0,0.000000,True,False,5-10 years,False,False,0.0,0,0.0
234,Others,Singapore,G,ACTIVE,Female,0,0,0,0,0,...,0.0,2.333333,True,True,6 months - 1 year,True,True,0.0,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5902,Chinese,Singapore,P,ACTIVE,Male,0,0,0,0,0,...,0.0,0.333333,True,True,6 months - 1 year,True,True,0.0,3,0.0
19152,Others,Singapore,G,ACTIVE,Female,0,0,0,0,0,...,0.0,0.000000,True,False,1-2 years,False,False,0.0,0,0.0
4238,Chinese,Singapore,P,ACTIVE,Male,0,0,0,0,0,...,0.0,0.000000,True,False,1-2 years,False,False,0.0,1,0.0
1929,Chinese,Singapore,P,ACTIVE,Male,0,0,0,0,0,...,0.0,0.142857,True,True,3-5 years,True,False,0.0,2,0.0


In [423]:
df2

,race_desc,ctrycode_desc,clttype,stat_flag,cltsex_fix,flg_is_revised_term,flg_is_rental_flat,flg_is_proposal,flg_with_preauthorisation,flg_is_returned_mail,...,avg_cancel_rate,Average_Bought_Gap,Exceed Average Gap,Exceed Average Gap with Purchase,Months_Last_Bought_Category,Claimed_Large_Amount,Lapsed_Large_Amount,Average_Lapse_Gap,f_hold,flg_hlthclaim_ever
18343,Others,Singapore,G,ACTIVE,Male,0,0,0,0,0,...,0.0,0.000000,True,False,5-10 years,True,False,0.0,2,0.0
19360,Chinese,Singapore,P,ACTIVE,Male,0,0,0,0,0,...,0.0,15.444444,False,False,<3 months,True,False,0.0,2,0.0
8680,Chinese,Others,P,ACTIVE,Female,0,0,0,0,0,...,0.0,0.000000,True,False,5-10 years,False,False,0.0,0,0.0
18227,Indian,Singapore,P,ACTIVE,Female,0,0,0,0,0,...,0.0,0.000000,True,False,5-10 years,False,False,0.0,0,0.0
1734,Chinese,Singapore,P,ACTIVE,Female,0,0,0,0,0,...,0.0,0.000000,True,False,5-10 years,False,False,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,Others,Singapore,P,LAPSED,Male,0,0,0,0,0,...,0.0,12.500000,True,True,2-3 years,False,False,0.0,0,0.0
11964,Chinese,Singapore,P,ACTIVE,Male,0,0,0,0,0,...,0.0,0.000000,True,False,> 10 years,True,False,0.0,1,0.0
5390,Chinese,Singapore,P,ACTIVE,Female,0,0,0,0,0,...,0.0,0.000000,True,False,5-10 years,False,False,0.0,0,0.0
860,Others,Singapore,P,ACTIVE,Male,0,0,0,0,0,...,0.0,37.000000,True,True,3-5 years,True,False,0.0,1,0.0


In [424]:
best_model.predict(df2)

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [417]:
joblib.dump(best_model, 'model_pipeline.pkl')

['model_pipeline.pkl']

In [435]:
def testing_hidden_data(hidden_data: pd.DataFrame) -> list:
    '''DO NOT REMOVE THIS FUNCTION.

The function accepts a dataframe as input and return an iterable (list)
of binary classes as output.

The function should be coded to test on hidden data
and should include any preprocessing functions needed for your model to perform. 
    
All relevant code MUST be included in this function.'''
    
    result = [] 
    df2 = clean_general_info(hidden_data)
    df2 = clean_up_risk_and_status(df2)
    df2 = clean_up_household_info(df2)
    df2 = clean_consent(df2)
    df2 = clean_policy(df2)
    df2 = clean_others(df2)
    df2 = f_hold_columns(df2)
    df2 = hlthclaim(df2)
    y_pred = best_model.predict(df2)
    result = y_pred.tolist()
    
    return result

In [436]:
data = pd.read_parquet(filepath).tail(200).drop('f_purchase_lh', axis=1)
testing_hidden_data(data)

[0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0